In [1]:
import time
start_time = time.time()

In [2]:
import pandas as pd 
import ast
from ast import literal_eval
import numpy as np
from pandas import DataFrame

#setting the file we want to read
file = "train.csv"

#reading from CSV
df = pd.read_csv(file, header= 0, quotechar='"', sep=',', encoding = "ISO-8859-1",keep_default_na=True, 
                dtype = { 'belongs_to_collection' : 'category',  'genres' : 'category', 'original_language' : 'category', 'production_companies' : 'category', 'production_countries' : 'category',  'spoken_languages' : 'category', 'Keywords' : 'category', 'cast' : 'category', 'crew' : 'category'},
                na_values = [0]
                )

#setting the list to be edited
dataList = [ 'belongs_to_collection',  'genres', 'production_companies', 'production_countries',  'spoken_languages', 'Keywords', 'cast', 'crew']
dataID = { 'belongs_to_collection' : 'id',  'genres': 'id', 'production_companies': 'id', 'production_countries': 'iso_3166_1',  'spoken_languages': 'iso_639_1', 'Keywords': 'id', 'cast': 'id', 'crew': 'id'}
dataNeeded = ['id', 'belongs_to_collection', 'budget', 'genres', 'original_language',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'runtime', 'spoken_languages', 'Keywords',
       'cast', 'crew', 'revenue']

#dropping the columns that was not needed
for col in df.keys():
    if not (col in dataNeeded) :
        df = df.drop(columns = col)

#filling empty list with []
for col in dataList :
    try:
        df[col] = df[col].cat.add_categories('[]',inplace = False)
    except:
        pass
    df[col].fillna( '[]' , inplace = True) 

#changing a sting of list into list
for col in dataList :
    df[[col]] = df[[col]].applymap(literal_eval)     

#changing the dict in the list into ID only
for col in dataList : 
    for row in range(len(df)) :
        for num in range(len(df[col][row])) :
            if not (isinstance(df[col][row][num], int) or isinstance(df[col][row][num], str)):
                df[col][row][num] = df[col][row][num][dataID[col]]    

# getting all unique values
uniqueSpokenLanguage = df['spoken_languages'].apply(pd.Series).stack().value_counts().keys().values.tolist()
for row in range(len(df)) :
    for num in range(len(df['spoken_languages'][row])):
        df['spoken_languages'][row][num] = uniqueSpokenLanguage.index(df.spoken_languages[row][num])

uniqueOrgLanguage = df['original_language'].unique().tolist()
df.original_language = df.original_language.replace(uniqueOrgLanguage,range(len(uniqueOrgLanguage)))

uniqueCountries = df['production_countries'].apply(pd.Series).stack().value_counts().keys().values.tolist()

for row in range(len(df)) :
    for num in range(len(df['production_countries'][row])):
        df['production_countries'][row][num] = uniqueCountries.index(df.production_countries[row][num])
print(time.time() - start_time)
df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,Keywords,cast,crew,revenue
0,1,[313576],14000000.0,[35],0,6.575393,"[4, 60, 8411]",[0],2/20/15,93.0,[0],"[4379, 9663, 11830, 179431]","[52997, 64342, 54729, 36801, 54812, 94098, 115...","[1449071, 3227, 347335, 347335, 57822, 1451395...",12314651
1,2,[107674],40000000.0,"[35, 18, 10751, 10749]",0,8.248895,[2],[0],8/6/2004,113.0,[0],"[2505, 4263, 6038, 13072]","[1813, 5823, 1210, 655, 33656, 62064, 68287, 1...","[1201, 1214, 4500, 8851, 12970, 38415, 38416, ...",95149435
2,3,[],3300000.0,[18],0,64.299990,"[2266, 3172, 32157]",[0],10/10/2014,105.0,[0],"[1416, 1523, 1640, 2176, 14512, 14819, 33896, ...","[996701, 18999, 129104, 970216, 223012, 159366...","[494, 6745, 9250, 23783, 23786, 53181, 52443, ...",13092000
3,4,[],1200000.0,"[53, 18]",1,3.174936,[],[5],3/9/2012,122.0,"[0, 8]","[10092, 10540, 11734, 14536, 14636, 208364, 22...","[35068, 85047, 1021524, 1093644, 86033, 92686,...","[95510, 95510, 95510]",16000000
4,5,[],NaN,"[28, 53]",2,1.148070,[],[15],2/5/2009,118.0,[11],[],"[84751, 64453, 84752, 1130534]","[84749, 84749]",3923970


In [23]:
# adding the extra columns for the most related crew
import multiprocessing
from multiprocessing import Pool
import CleaningFunctions
from functools import partial
from IPython.display import display, clear_output
import math

start_time = time.time()

if __name__ ==  '__main__': 
  
    iterable = dataList
    pool = multiprocessing.Pool(len(dataList))
    func = partial(CleaningFunctions.Getting3MostRelated, df)
    df_temp = pool.map(func, iterable)
    pool.close()
    pool.join()
    print(time.time() - start_time)
    
for bigCol in range(len(df_temp)):
    for smallCol in df_temp[bigCol][0]:
        df[smallCol] = df_temp[bigCol][0][smallCol]

62.00406312942505


In [24]:
df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,...,spoken_languages2,Keywords0,Keywords1,Keywords2,cast0,cast1,cast2,crew0,crew1,crew2
0,1,[313576],14000000.0,[35],0,6.575393,"[4, 60, 8411]",[0],21964.75,93.0,...,NaN,4379.0,179431.0,9663.0,94098.0,98879.0,59256.0,1449071.0,1393375.0,1431552.0
1,2,[107674],40000000.0,"[35, 18, 10751, 10749]",0,8.248895,[2],[0],18114.75,113.0,...,NaN,2505.0,4263.0,6038.0,1665036.0,303542.0,62064.0,1214.0,38416.0,38415.0
2,3,[],3300000.0,[18],0,64.299990,"[2266, 3172, 32157]",[0],21831.75,105.0,...,NaN,1416.0,206298.0,1523.0,16506.0,52939.0,15824.0,1402475.0,1049456.0,1406825.0
3,4,[],1200000.0,"[53, 18]",1,3.174936,[],[5],20886.75,122.0,...,NaN,11734.0,10540.0,14636.0,35068.0,85047.0,1021524.0,95510.0,NaN,NaN
4,5,[],NaN,"[28, 53]",2,1.148070,[],[15],19758.75,118.0,...,NaN,NaN,NaN,NaN,84751.0,64453.0,84752.0,84749.0,NaN,NaN


In [9]:
#changing the time into numerical value
start_time = time.time()

if __name__ ==  '__main__': 
  
    iterable = range(len(df))
    pool = multiprocessing.Pool(len(dataList))
    func = partial(CleaningFunctions.time2num, df)
    result = pool.map(func, iterable)
    pool.close()
    pool.join()
    print(time.time() - start_time)

df["release_date"] = result

76.98667740821838


In [26]:
# save to save.csv

df.to_csv('save_Cleaning.csv',index_label = False)
df = pd.read_csv('save_Cleaning.csv')

In [27]:
#dropping the extra columns that was already extracted
for col in dataList:
    if(df[col].bool):
        df = df.drop(columns=col)
df.head()

,id,budget,original_language,popularity,release_date,runtime,revenue,belongs_to_collection0,belongs_to_collection1,belongs_to_collection2,...,spoken_languages2,Keywords0,Keywords1,Keywords2,cast0,cast1,cast2,crew0,crew1,crew2
0,1,14000000.0,0,6.575393,21964.75,93.0,12314651,313576.0,NaN,NaN,...,NaN,4379.0,179431.0,9663.0,94098.0,98879.0,59256.0,1449071.0,1393375.0,1431552.0
1,2,40000000.0,0,8.248895,18114.75,113.0,95149435,107674.0,NaN,NaN,...,NaN,2505.0,4263.0,6038.0,1665036.0,303542.0,62064.0,1214.0,38416.0,38415.0
2,3,3300000.0,0,64.299990,21831.75,105.0,13092000,NaN,NaN,NaN,...,NaN,1416.0,206298.0,1523.0,16506.0,52939.0,15824.0,1402475.0,1049456.0,1406825.0
3,4,1200000.0,1,3.174936,20886.75,122.0,16000000,NaN,NaN,NaN,...,NaN,11734.0,10540.0,14636.0,35068.0,85047.0,1021524.0,95510.0,NaN,NaN
4,5,NaN,2,1.148070,19758.75,118.0,3923970,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,84751.0,64453.0,84752.0,84749.0,NaN,NaN


In [28]:
# counting the unique value for each col (idk why i am doing this too lol)
unique_count = {}
for col in df:
    count = df[col].apply(pd.Series).stack().value_counts()
    unique_count.update({col: count})

In [30]:
#creating df df_pred to do prediction for those with non-nan values > 2000
predicting_list = []
for col in df:
    print(col,df[col].count())
    if (df[col].count() > 2000):
        predicting_list.append(col)
predicting_list.remove("revenue")


df_pred = pd.DataFrame()
df_pred = df[predicting_list].copy()

id 3000
budget 2188
original_language 3000
popularity 3000
release_date 3000
runtime 2986
revenue 3000
belongs_to_collection0 604
belongs_to_collection1 0
belongs_to_collection2 0
genres0 2993
genres1 2400
genres2 1428
production_companies0 2844
production_companies1 2069
production_companies2 1335
production_countries0 2945
production_countries1 723
production_countries2 198
spoken_languages0 2980
spoken_languages1 875
spoken_languages2 326
Keywords0 2724
Keywords1 2537
Keywords2 2330
cast0 2974
cast1 2958
cast2 2950
crew0 2984
crew1 2840
crew2 2668


In [32]:
#cause knn dosent accept a string heading, so changng the heading
df_pred.columns = range(df_pred.shape[1])
df_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1,14000000.0,0,6.575393,21964.75,93.0,35.0,NaN,60.0,8411.0,...,0.0,4379.0,179431.0,9663.0,94098.0,98879.0,59256.0,1449071.0,1393375.0,1431552.0
1,2,40000000.0,0,8.248895,18114.75,113.0,18.0,10749.0,2.0,NaN,...,0.0,2505.0,4263.0,6038.0,1665036.0,303542.0,62064.0,1214.0,38416.0,38415.0
2,3,3300000.0,0,64.299990,21831.75,105.0,18.0,NaN,2266.0,32157.0,...,0.0,1416.0,206298.0,1523.0,16506.0,52939.0,15824.0,1402475.0,1049456.0,1406825.0
3,4,1200000.0,1,3.174936,20886.75,122.0,18.0,53.0,NaN,NaN,...,8.0,11734.0,10540.0,14636.0,35068.0,85047.0,1021524.0,95510.0,NaN,NaN
4,5,NaN,2,1.148070,19758.75,118.0,53.0,28.0,NaN,NaN,...,11.0,NaN,NaN,NaN,84751.0,64453.0,84752.0,84749.0,NaN,NaN


In [297]:
#checking if everything is in float for df
for x in df:
    for y in range(len(df)):
        if not (isinstance(df[x][y], float) or isinstance(df[x][y], int) or math.isnan(float(df_copy[x][y]))):
            print(x,y,df[x][y],type(df[x][y]))

Keywords0 0 4379 <class 'str'>
Keywords0 1 2505 <class 'str'>
Keywords0 2 1416 <class 'str'>
Keywords0 3 11734 <class 'str'>
Keywords0 7 5939 <class 'str'>
Keywords0 8 6300 <class 'str'>
Keywords0 9 11800 <class 'str'>
Keywords0 10 700 <class 'str'>
Keywords0 11 6876 <class 'str'>
Keywords0 12 2483 <class 'str'>
Keywords0 13 4129 <class 'str'>
Keywords0 14 476 <class 'str'>
Keywords0 15 379 <class 'str'>
Keywords0 16 2708 <class 'str'>
Keywords0 17 5565 <class 'str'>
Keywords0 18 3522 <class 'str'>
Keywords0 19 1930 <class 'str'>
Keywords0 20 4379 <class 'str'>
Keywords0 21 12339 <class 'str'>
Keywords0 22 171993 <class 'str'>
Keywords0 24 2658 <class 'str'>
Keywords0 26 811 <class 'str'>
Keywords0 27 10556 <class 'str'>
Keywords0 28 4290 <class 'str'>
Keywords0 29 4056 <class 'str'>
Keywords0 30 6054 <class 'str'>
Keywords0 31 6258 <class 'str'>
Keywords0 32 177900 <class 'str'>
Keywords0 33 210012 <class 'str'>
Keywords0 36 6167 <class 'str'>
Keywords0 37 1261 <class 'str'>
Keywords0

ValueError: could not convert string to float: '[]'

In [33]:
# finally doing the prediction lol ==" i want to cry
from fancyimpute import KNN 
df_pred = pd.DataFrame(KNN(k=5).fit_transform(df_pred))

Using TensorFlow backend.


Imputing row 1/3000 with 1 missing, elapsed time: 1.882
Imputing row 101/3000 with 0 missing, elapsed time: 1.887
Imputing row 201/3000 with 0 missing, elapsed time: 1.892
Imputing row 301/3000 with 0 missing, elapsed time: 1.896
Imputing row 401/3000 with 0 missing, elapsed time: 1.900
Imputing row 501/3000 with 1 missing, elapsed time: 1.905
Imputing row 601/3000 with 1 missing, elapsed time: 1.910
Imputing row 701/3000 with 0 missing, elapsed time: 1.914
Imputing row 801/3000 with 0 missing, elapsed time: 1.917
Imputing row 901/3000 with 0 missing, elapsed time: 1.922
Imputing row 1001/3000 with 2 missing, elapsed time: 1.927
Imputing row 1101/3000 with 3 missing, elapsed time: 1.930
Imputing row 1201/3000 with 2 missing, elapsed time: 1.934
Imputing row 1301/3000 with 1 missing, elapsed time: 1.938
Imputing row 1401/3000 with 1 missing, elapsed time: 1.942
Imputing row 1501/3000 with 0 missing, elapsed time: 1.946
Imputing row 1601/3000 with 0 missing, elapsed time: 1.950
Imputing 

In [35]:
#adding back the columns for df_pred
df_pred.columns = predicting_list

In [37]:
#merging df with df_pred
for col in df_pred:
    df[col] = df_pred[col]

In [38]:
df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,...,spoken_languages2,Keywords0,Keywords1,Keywords2,cast0,cast1,cast2,crew0,crew1,crew2
0,1.0,[313576],1.400000e+07,[35],0.0,6.575393,"[4, 60, 8411]",[0],21964.75,93.0,...,NaN,4379.000000,179431.000000,9663.000000,94098.0,98879.0,59256.0,1449071.0,1.393375e+06,1.431552e+06
1,2.0,[107674],4.000000e+07,"[35, 18, 10751, 10749]",0.0,8.248895,[2],[0],18114.75,113.0,...,NaN,2505.000000,4263.000000,6038.000000,1665036.0,303542.0,62064.0,1214.0,3.841600e+04,3.841500e+04
2,3.0,[],3.300000e+06,[18],0.0,64.299990,"[2266, 3172, 32157]",[0],21831.75,105.0,...,NaN,1416.000000,206298.000000,1523.000000,16506.0,52939.0,15824.0,1402475.0,1.049456e+06,1.406825e+06
3,4.0,[],1.200000e+06,"[53, 18]",1.0,3.174936,[],[5],20886.75,122.0,...,NaN,11734.000000,10540.000000,14636.000000,35068.0,85047.0,1021524.0,95510.0,4.038257e+05,2.099745e+05
4,5.0,[],1.556856e+08,"[28, 53]",2.0,1.148070,[],[15],19758.75,118.0,...,NaN,33855.439794,8551.623303,4671.385949,84751.0,64453.0,84752.0,84749.0,6.776041e+04,6.030036e+05
